In [ ]:
import numpy as np

import xarray as xr
from matplotlib import pyplot as plt
%matplotlib inline

from xhistogram.xarray import histogram
import dask.dataframe as dd
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import mitequinox.utils as ut
from mitequinox.plot import *
import mitequinox.parcels as pa
from xmitgcm import llcreader

from fsspec.implementations.local import LocalFileSystem

In [ ]:
from dask.distributed import Client, LocalCluster
#
cluster = LocalCluster()
#
from dask_jobqueue import PBSCluster
#cluster = PBSCluster() #processes=7, cores=7
#w = cluster.scale(jobs=4)
#
client = Client(cluster)

In [ ]:
#Eulerian data
dsU =  xr.open_zarr(ut.work_data_dir+'rechunked/SSU_rot.zarr',consolidated=False)
dsV =  xr.open_zarr(ut.work_data_dir+'rechunked/SSV_rot.zarr',consolidated=False)

In [ ]:
dsU_mean = dsU.mean('time').persist()

In [ ]:
dsV_mean = dsV.mean('time').persist()

In [ ]:
face,i,j = 4, 2000, 2000
dsi = dsU.isel(face=face,i=i,j=i).persist()

### Figures Eulerian data

In [ ]:
fig,ax = plt.subplots(figsize=(7,5))
dsi.SSU_rot.plot(ax=ax)

In [ ]:
# Map of mean SSU
pl.plot_pretty(dsU_mean.SSU_rot,cmap='magma',vmax=0.1,land=False)

In [ ]:
# Map of mean SSV
pl.plot_pretty(dsU_mean.SSV_rot,cmap='magma',vmax=0.1,land=False)

In [ ]:
# Lagrangian data
root_dir = '/home/datawork-lops-osi/equinox/mit4320/parcels/'
run_name = 'global_extra_T365j_dt1j_dij50'
p = pa.parcels_output(root_dir+run_name, parquets=['trajectory'])
df = p.df['trajectory']
df = pa.degs2ms(df) #Convert in m/s 


### Figures Lagrangian data

In [ ]:
cluster.close()